In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression 
import pandas as pd
import numpy as np
import datetime
import os

print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Predictive_Model_Building/model_building
2020-05-24 00:12:24.168077


In [2]:
df_input_all=pd.read_csv("./date_0831_updated_with_competitors_JL_2020-05-15.csv")
print(df_input_all.shape)
df_input_all.head(2)

(577766, 453)


,customer_id_hashed,DV_cumulative_week_1,DV_cumulative_week_2,DV_cumulative_week_3,DV_cumulative_week_4,weeks_since_sign_up,P_zip,S_zip,else_10_zip,signed_online,...,Furniture Stores_Furniture Stores,Grocery Stores_Convenience Stores,Grocery Stores_Supermarkets and Other Grocery (except Convenience) Stores,Home Furnishings Stores_All Other Home Furnishings Stores,"Lawn and Garden Equipment and Supplies Stores_Nursery, Garden Center, and Farm Supply Stores","Office Supplies, Stationery, and Gift Stores_Gift, Novelty, and Souvenir Stores",Other Miscellaneous Store Retailers_All Other Miscellaneous Store Retailers (except Tobacco Stores),Other Miscellaneous Store Retailers_Pet and Pet Supplies Stores,Specialty Food Stores_All Other Specialty Food Stores,"Sporting Goods, Hobby, and Musical Instrument Stores_Hobby, Toy, and Game Stores"
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,0,0,0,0,49,1,1,0,0,...,104.0,65.0,319.0,47.0,0.0,65.0,16.0,76.0,17.0,45.0
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,0,0,0,0,469,1,1,0,0,...,22.0,34.0,77.0,15.0,2.0,18.0,6.0,28.0,0.0,7.0


In [3]:
# revise dvs

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
list_files_POS=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
list_files_POS=[x for x in list_files_POS if "dailysales" in x.lower()]
list_files_POS=[x for x in list_files_POS if x.split("s/MediaStorm_")[1][:10]>="2019-09-01"]
list_files_POS.sort()
list_files_POS=list_files_POS[:4]
list_files_POS

['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-07/MediaStormDailySales20190910-120643-163.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-14/MediaStormDailySales20190917-110425-322.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-21/MediaStormDailySales20190924-112126-506.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-09-28/MediaStormDailySales20191001-111241-312.txt']

In [4]:
week_i=1
df_sale_dv=df_input_all[['customer_id_hashed']]
for file in list_files_POS:
    df=pd.read_csv(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df=df.groupby(['location_id','transaction_dt','transaction_id','customer_id_hashed'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df[df['item_transaction_amt']>0]
    df=df[['customer_id_hashed']].drop_duplicates()
    df["DV_single_week_"+str(week_i)]=1
    df_sale_dv=pd.merge(df_sale_dv,df,on="customer_id_hashed",how="left")
    
    print(week_i,datetime.datetime.now())
    week_i+=1
df_sale_dv=df_sale_dv.fillna(0)
df_sale_dv.head(2)

1 2020-05-24 00:13:31.847920
2 2020-05-24 00:14:17.833729
3 2020-05-24 00:15:02.004019
4 2020-05-24 00:15:55.682296


,customer_id_hashed,DV_single_week_1,DV_single_week_2,DV_single_week_3,DV_single_week_4
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,0.0,0.0,0.0,0.0
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,0.0,0.0,0.0,0.0


In [5]:
a=[]
for i in range(len(list_files_POS)):
    i_int_pos=i+1
    list_cum_i=[]
    for j in range(i_int_pos):
        list_cum_i.append('DV_single_week_'+str(j+1))
    a.append(list_cum_i)
    df_sale_dv['DV_cumulative_week_updated_'+str(i_int_pos)]=df_sale_dv[list_cum_i].sum(axis=1)
    df_sale_dv['DV_cumulative_week_updated_'+str(i_int_pos)]=np.where(df_sale_dv['DV_cumulative_week_updated_'+str(i_int_pos)]>0,1,0)
    

In [6]:
list_cols_to_delete=['week_diff','DV_cumulative_week_1','DV_cumulative_week_2','DV_cumulative_week_3','DV_cumulative_week_4',
                     'department_minus_one_trans','total_units_department_minus_1_1st_trans','total_units_department_minus_1_1st_trans_recent_one',
                    'total_units_department_minus_1_recent_2_trans','total_units_department_minus_1_recent_3_trans',
                    'total_units_department_minus_1_recent_4_trans','total_units_department_minus_1_recent_5_trans']
for col in list_cols_to_delete:
    del df_input_all[col]

df_input_all=pd.merge(df_sale_dv,df_input_all,on="customer_id_hashed",how="right")
df_input_all['DV_cumulative_week_updated_1']=df_input_all['DV_cumulative_week_updated_1'].fillna(0)
df_input_all['DV_cumulative_week_updated_2']=df_input_all['DV_cumulative_week_updated_2'].fillna(0)
df_input_all['DV_cumulative_week_updated_3']=df_input_all['DV_cumulative_week_updated_3'].fillna(0)
df_input_all['DV_cumulative_week_updated_4']=df_input_all['DV_cumulative_week_updated_4'].fillna(0)


In [7]:
df_input_all.to_csv("./Data_tabel_with_DCM_20190831_updated_with_competitors_and_DVs_JL_2020-05-23.csv")